In [ ]:
#imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import gradio as gr 


openai_api_key = os.getenv('OPENAI_API_KEY')
#anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

#load environment variables
load_dotenv(override=True)

#using openai library 
#initialize openai client and set api keys for other providers

openAI = OpenAI()

#anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"
grok_url = "https://api.x.ai/v1"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

#Initialize clients for each provider using openai library

#anthropic = OpeanAi(api_key=anthropic_api_key,url=anthropic_url)
gemini = OpenAI(api_key=google_api_key,base_url=gemini_url)
groq = OpenAI(api_key=groq_api_key,base_url=groq_url)
grok = OpenAI(api_key=grok_api_key,base_url=grok_url)
openrouter = OpenAI(api_key=openrouter_api_key,base_url=openrouter_url)
ollama = OpenAI(api_key="ollama",base_url=ollama_url)

In [ ]:
hard = """
On a bookshelf, two volumes of Pushkin stand side by side: the first and the second.
The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick.
A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume.
What distance did it gnaw through?
"""
hard_puzzle = [
    {"role": "user", "content": hard}
]

In [ ]:
#!ollama pull gpt-oss:20b

In [ ]:
#response = ollama.chat.completions.create(model="gpt-oss:20b", messages=hard_puzzle)
#display(Markdown(response.choices[0].message.content))

In [ ]:
#example ollama

response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=hard_puzzle)
display(Markdown(response.choices[0].message.content))

#response = openai.chat.completions.create(model="gpt-5-nano", messages=hard_puzzle, reasoning_effort="minimal")
#display(Markdown(response.choices[0].message.content))

In [ ]:
response = openrouter.chat.completions.create(model="xiaomi/mimo-v2-flash:free", messages=hard_puzzle)
display(Markdown(response.choices[0].message.content))

In [ ]:
tell_a_joke = [
    {"role": "user", "content": "Tell a joke for a student on the journey to becoming an expert in LLM Engineering"},
]

##Using Langchain with gemini model

To be able to change the model to gemini we had to install the package langchain_google_genai as code below.
I have faced some issues to install the library in the correct enviorment .venv (script) running on this kernel
Below also the code I got with AI aid
There were also no pip and I have to run the following on bash ensuring pip is available
c:\ProjetosGit\llm_engineering\.venv\Scripts\python.exe -m ensurepip --upgrade

In [ ]:
import sys
print(sys.executable)


In [ ]:
!pip show langchain-google-genai


In [ ]:
%pip install -U langchain-google-genai

In [ ]:
import sys
!{sys.executable} -m pip install -U langchain-google-genai


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain_googl
#from langchain_openai import ChatOpenAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

response = llm.invoke(tell_a_joke)

display(Markdown(response.content))

## Litte LLM remarks

I had to instal google cloud cli to the computer for authentification purposes
To Bypass Vertex Project id put "gemini/" before the selected model
The code exemple was on day1.ipynb just noticed after struggling. Have to check if this authentification would be need anyway

In [ ]:
from litellm import completion
import os

os.environ['GOOLGE_API_KEY'] = google_api_key
response = completion(
    model="gemini/gemini-2.5-flash", 
    messages=tell_a_joke    
    
)
reply = response.choices[0].message.content
display(Markdown(reply))

In [ ]:
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")


In [ ]:
with open("hamlet.txt", "r", encoding="utf-8") as f:
    hamlet = f.read()

loc = hamlet.find("Speak, man")
print(hamlet[loc:loc+100])

In [ ]:
question = [{"role": "user", "content": "In Hamlet, when Laertes asks 'Where is my father?' what is the reply?"}]

In [ ]:
response = completion(model="gemini/gemini-2.5-flash-lite", messages=question)
display(Markdown(response.choices[0].message.content))

In [ ]:
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")

In [ ]:
question[0]["content"] += "\n\nFor context, here is the entire text of Hamlet:\n\n"+hamlet

In [ ]:
response = completion(model="gemini/gemini-2.5-flash-lite", messages=question)
display(Markdown(response.choices[0].message.content))

In [ ]:
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")

#Now the Chat implementation

In [ ]:
# Let's make a conversation between gemini-2.5-flash-lite and xiaomi/mimo-v2-flash:free (just got one free on openrouter)
# We're using cheap versions of models so the costs will be minimal

gemini_model = "gemini-2.5-flash"
xiomi_model = "xiaomi/mimo-v2-flash:free"

gemini_system = "You are a chatbot who is very argumentative; \
old drilling seals sargent, and thinks is still a marine and not an assistant."

xiomi_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gemini_messages = ["Hi there"]
xiomi_messages = ["Hi"]

In [ ]:
#Function talk and talkback, passing model, model instance, outgoing message, incoming message and model_system definition


def call_model_2_talk(model_system,model_inst,model,model_messages,incoming_messages):
    messages=[{"role": "system" ,"content":model_system}]
    for outmessage, inmessage in zip(model_messages,incoming_messages):
        messages.append({"role":"assistant","content":outmessage})
        messages.append({"role":"user","content":inmessage})
    response = model_inst.chat.completions.create(model =model,messages=messages)
    return response.choices[0].message.content
    

def call_model_2_talkback(model_system,model_inst,model,model_messages,incoming_messages):
    messages = [{"role": "system", "content": model_system}]
    for outmessage, inmessage in zip(model_messages, incoming_messages):
        messages.append({"role": "user", "content": outmessage})
        messages.append({"role": "assistant", "content": inmessage})
    messages.append({"role": "user", "content": model_messages[-1]})
    response = model_inst.chat.completions.create(model=model, messages=messages)
    return response.choices[0].message.content

In [ ]:
call_model_2_talk(gemini_system,gemini,gemini_model,gemini_messages,xiomi_messages)

In [ ]:
call_model_2_talkback(xiomi_system,openrouter,xiomi_model,xiomi_messages,gemini_messages)

In [ ]:


gemini_messages = ["Hi there"]
xiomi_messages = ["Hi"]

display(Markdown(f"### Gemini:\n{gemini_messages[0]}\n"))
display(Markdown(f"### Xiomi:\n{xiomi_messages[0]}\n"))

for i in range(5):
    gemini_next = call_model_2_talk(gemini_system,gemini,gemini_model,gemini_messages,xiomi_messages)
    display(Markdown(f"### Gemini:\n{gemini_next}\n"))
    gemini_messages.append(gemini_next)
    
    xiomi_next = call_model_2_talkback(xiomi_system,openrouter,xiomi_model,xiomi_messages,gemini_messages)
    display(Markdown(f"### Xiomi:\n{xiomi_next}\n"))
    xiomi_messages.append(xiomi_next)

In [ ]:
system_message = "You are a helpful assistant"

def message_gemini(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = gemini.chat.completions.create(model=gemini_model, messages=messages)
    return response.choices[0].message.content



In [ ]:
message_gemini("What is today's date?")

In [ ]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
shout("Hello world")

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("teste", "teste123"))

In [ ]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
# Adding a little more:

message_input = gr.Textbox(label="Your message:", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=shout,
    title="Shout", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for Gemini", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=message_gemini,
    title="Gemini", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
# Let's use Markdown
# Are you wondering why it makes any difference to set system_message when it's not referred to in the code below it?
# I'm taking advantage of system_message being a global variable, used back in the message_gpt function (go take a look)
# Not a great software engineering practice, but quite common during Jupyter Lab R&D!

system_message = "You are a helpful assistant that responds in markdown without code blocks"

message_input = gr.Textbox(label="Your message:", info="Enter a message for Gemini Flash", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=message_gemini,
    title="GEMINI", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python guide in the guides folder

def stream_gemini(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = gemini.chat.completions.create(
        model='gemini-2.5-flash',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_gemini,
    title="Gemini", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
def stream_xiomi(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openrouter.chat.completions.create(
        model='xiaomi/mimo-v2-flash:free',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
def stream_model(prompt, model):
    if model=="Gemini":
        result = stream_gemini(prompt)
    elif model=="Xiomi":
        result = stream_xiomi(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["Gemini", "Xiomi"], label="Select model", value="Gemini")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    title="LLMs", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Explain the Transformer architecture to a layperson", "GPT"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "Claude"]
        ], 
    flagging_mode="never"
    )
view.launch()

In [ ]:
from scraper import fetch_website_contents

In [ ]:
# Again this is typical Experimental mindset - I'm changing the global variable we used above:

system_message = """
You are an assistant that analyzes the contents of a company website landing page
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

In [ ]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += fetch_website_contents(url)
    if model=="Gemini":
        result = stream_gemini(prompt)
    elif model=="Xiomi":
        result = stream_xiomi(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
name_input = gr.Textbox(label="Company name:")
url_input = gr.Textbox(label="Landing page URL including http:// or https://")
model_selector = gr.Dropdown(["Gemini", "Xiomi"], label="Select model", value="Gemini")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_brochure,
    title="Brochure Generator", 
    inputs=[name_input, url_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Hugging Face", "https://huggingface.co", "GPT"],
            ["Edward Donner", "https://edwarddonner.com", "Claude"]
        ], 
    flagging_mode="never"
    )
view.launch()